In [0]:
!pip install tensorflow-gpu
!pip install grpcio==1.25

In [8]:
import os
import tensorflow as tf
print("Tensorflow version: ",tf.__version__)
# tf.debugging.set_log_device_placement(True)


string = ('data', '/midi')
s = ''
dir = s.join(string)

#creating data folder
if not os.path.isdir(dir):
  os.makedirs(dir)
  print('Created directory {}'.format(dir))

#creating experiments folder
if not os.path.isdir("experiments"):
  os.makedirs("experiments")
  print('Created directory experiments')


!ls

Tensorflow version:  2.0.0
data  experiments  sample_data


In [0]:
#@title Run Utility functions (double click to open) { form-width: "15%", display-mode: "form" }

import os, glob, random
import pretty_midi
import numpy as np
from tensorflow.keras.models import model_from_json
from multiprocessing import Pool as ThreadPool

def log(message, verbose):
	if verbose:
		print('[*] {}'.format(message))

def parse_midi(path):
    midi = None
    try:
        midi = pretty_midi.PrettyMIDI(path)
        midi.remove_invalid_notes()
    except Exception as e:
        raise Exception(("%s\nerror readying midi file %s" % (e, path)))
    return midi

def get_percent_monophonic(pm_instrument_roll):
    mask = pm_instrument_roll.T > 0
    notes = np.sum(mask, axis=1)
    n = np.count_nonzero(notes)
    single = np.count_nonzero(notes == 1)
    if single > 0:
        return float(single) / float(n)
    elif single == 0 and n > 0:
        return 0.0
    else: # no notes of any kind
        return 0.0
    
def filter_monophonic(pm_instruments, percent_monophonic=0.99):
    return [i for i in pm_instruments if \
            get_percent_monophonic(i.get_piano_roll()) >= percent_monophonic]


# if the experiment dir doesn't exist create it and its subfolders
def create_experiment_dir(experiment_dir, verbose=False):
    
    # if the experiment directory was specified and already exists
    if experiment_dir != 'experiments/default' and \
       os.path.exists(experiment_dir):
    	# raise an error
    	raise Exception('Error: Invalid --experiemnt_dir, {} already exists' \
    		            .format(experiment_dir))

    # if the experiment directory was not specified, create a new numeric folder
    if experiment_dir == 'experiments/default':
    	
    	experiments = os.listdir('experiments')
    	experiments = [dir_ for dir_ in experiments \
    	               if os.path.isdir(os.path.join('experiments', dir_))]
    	
    	most_recent_exp = 0
    	for dir_ in experiments:
    		try:
    			most_recent_exp = max(int(dir_), most_recent_exp)
    		except ValueError as e:
    			# ignrore non-numeric folders in experiments/
    			pass

    	experiment_dir = os.path.join('experiments', 
    		                          str(most_recent_exp + 1).rjust(2, '0'))

    os.mkdir(experiment_dir)
    log('Created experiment directory {}'.format(experiment_dir), verbose)
    os.mkdir(os.path.join(experiment_dir, 'checkpoints'))
    log('Created checkpoint directory {}'.format(os.path.join(experiment_dir, 'checkpoints')),
    	verbose)
    os.mkdir(os.path.join(experiment_dir, 'tensorboard-logs'))
    log('Created log directory {}'.format(os.path.join(experiment_dir, 'tensorboard-logs')), 
    	verbose)

    return experiment_dir

# load data with a lazzy loader
def get_data_generator(midi_paths, 
                       window_size=20, 
                       batch_size=32,
                       num_threads=8,
                       max_files_in_ram=170):

    if num_threads > 1:
    	# load midi data
    	pool = ThreadPool(num_threads)

    load_index = 0

    while True:
        load_files = midi_paths[load_index:load_index + max_files_in_ram]
        # print('length of load files: {}'.format(len(load_files)))
        load_index = (load_index + max_files_in_ram) % len(midi_paths)

        # print('loading large batch: {}'.format(max_files_in_ram))
        # print('Parsing midi files...')
        # start_time = time.time()
        if num_threads > 1:
       		parsed = pool.map(parse_midi, load_files)
       	else:
       		parsed = map(parse_midi, load_files)
        # print('Finished in {:.2f} seconds'.format(time.time() - start_time))
        # print('parsed, now extracting data')
        data = _windows_from_monophonic_instruments(parsed, window_size)
        batch_index = 0
        while batch_index + batch_size < len(data[0]):
            # print('getting data...')
            # print('yielding small batch: {}'.format(batch_size))
            
            res = (data[0][batch_index: batch_index + batch_size], 
                   data[1][batch_index: batch_index + batch_size])
            yield res
            batch_index = batch_index + batch_size
        
        # probably unneeded but why not
        del parsed # free the mem
        del data # free the mem

def save_model(model, model_dir):
    with open(os.path.join(model_dir, 'model.json'), 'w') as f:
        f.write(model.to_json())

def load_model_from_checkpoint(model_dir):

    '''Loads the best performing model from checkpoint_dir'''
    with open(os.path.join(model_dir, 'model.json'), 'r') as f:
        model = model_from_json(f.read())

    epoch = 0
    newest_checkpoint = max(glob.iglob(model_dir + 
    	                    '/checkpoints/*.hdf5'), 
                            key=os.path.getctime)
    print(newest_checkpoint)
    if newest_checkpoint: 
      #  epoch = int(newest_checkpoint[-22:-19])
       epoch = 1
       model.load_weights(newest_checkpoint)

    return model, epoch

def generate(model, seeds, window_size, length, num_to_gen, instrument_name):
    
    # generate a pretty midi file from a model using a seed
    def _gen(model, seed, window_size, length):
        
        generated = []
        # ring buffer
        buf = np.copy(seed).tolist()
        while len(generated) < length:
            arr = np.expand_dims(np.asarray(buf), 0)
            pred = model.predict(arr)
            
            # argmax sampling (NOT RECOMMENDED), or...
            # index = np.argmax(pred)
            
            # prob distrobuition sampling
            index = np.random.choice(range(0, seed.shape[1]), p=pred[0])
            pred = np.zeros(seed.shape[1])

            pred[index] = 1
            generated.append(pred)
            buf.pop(0)
            buf.append(pred)

        return generated

    midis = []
    for i in range(0, num_to_gen):
        seed = seeds[random.randint(0, len(seeds) - 1)]
        gen = _gen(model, seed, window_size, length)
        midis.append(_network_output_to_midi(gen, instrument_name))
    return midis

# create a pretty midi file with a single instrument using the one-hot encoding
# output of keras model.predict.
def _network_output_to_midi(windows, 
                           instrument_name='Acoustic Grand Piano', 
                           allow_represses=False):

    # Create a PrettyMIDI object
    midi = pretty_midi.PrettyMIDI()
    # Create an Instrument instance for a cello instrument
    instrument_program = pretty_midi.instrument_name_to_program(instrument_name)
    instrument = pretty_midi.Instrument(program=instrument_program)
    
    cur_note = None # an invalid note to start with
    cur_note_start = None
    clock = 0

    # Iterate over note names, which will be converted to note number later
    for step in windows:

        note_num = np.argmax(step) - 1
        
        # a note has changed
        if allow_represses or note_num != cur_note:
            
            # if a note has been played before and it wasn't a rest
            if cur_note is not None and cur_note >= 0:            
                # add the last note, now that we have its end time
                note = pretty_midi.Note(velocity=127, 
                                        pitch=int(cur_note), 
                                        start=cur_note_start, 
                                        end=clock)
                instrument.notes.append(note)

            # update the current note
            cur_note = note_num
            cur_note_start = clock

        # update the clock
        clock = clock + 1.0 / 4

    # Add the cello instrument to the PrettyMIDI object
    midi.instruments.append(instrument)
    return midi

# returns X, y data windows from all monophonic instrument
# tracks in a pretty midi file
def _windows_from_monophonic_instruments(midi, window_size):
    X, y = [], []
    for m in midi:
        if m is not None:
            melody_instruments = filter_monophonic(m.instruments, 1.0)
            for instrument in melody_instruments:
                if len(instrument.notes) > window_size:
                    windows = _encode_sliding_windows(instrument, window_size)
                    for w in windows:
                        X.append(w[0])
                        y.append(w[1])
    return (np.asarray(X), np.asarray(y))

# one-hot encode a sliding window of notes from a pretty midi instrument.
# This approach uses the piano roll method, where each step in the sliding
# window represents a constant unit of time (fs=4, or 1 sec / 4 = 250ms).
# This allows us to encode rests.
# expects pm_instrument to be monophonic.
def _encode_sliding_windows(pm_instrument, window_size):
    
    roll = np.copy(pm_instrument.get_piano_roll(fs=4).T)

    # trim beginning silence
    summed = np.sum(roll, axis=1)
    mask = (summed > 0).astype(float)
    roll = roll[np.argmax(mask):]
    
    # transform note velocities into 1s
    roll = (roll > 0).astype(float)
    
    # calculate the percentage of the events that are rests
    # s = np.sum(roll, axis=1)
    # num_silence = len(np.where(s == 0)[0])
    # print('{}/{} {:.2f} events are rests'.format(num_silence, len(roll), float(num_silence)/float(len(roll))))

    # append a feature: 1 to rests and 0 to notes
    rests = np.sum(roll, axis=1)
    rests = (rests != 1).astype(float)
    roll = np.insert(roll, 0, rests, axis=1)
    
    windows = []
    for i in range(0, roll.shape[0] - window_size - 1):
        windows.append((roll[i:i + window_size], roll[i + window_size + 1]))
    return windows

# Model Training

In [0]:
import os, argparse, time
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax, Nadam

OUTPUT_SIZE = 129 # 0-127 notes + 1 for rests

# create or load a saved model
# returns the model and the epoch number (>1 if loaded from checkpoint)

def get_model(experiment_dir=None):
    
    epoch = 0
    num_layers = 1
    rnn_size = 64
    window_size = 20
    dropout = 0.2
    grad_clip = 5
    learning_rate = None
    optimizer = "adam"
    if not experiment_dir:
        model = Sequential()
        for layer_index in range(num_layers):
            kwargs = dict() 
            kwargs['units'] = rnn_size
            # if this is the first layer
            if layer_index == 0:
                kwargs['input_shape'] = (window_size, OUTPUT_SIZE)
                if num_layers == 1:
                    kwargs['return_sequences'] = False
                else:
                    kwargs['return_sequences'] = True
                model.add(LSTM(**kwargs))
            else:
                # if this is a middle layer
                if not layer_index == num_layers - 1:
                    kwargs['return_sequences'] = True
                    model.add(LSTM(**kwargs))
                else: # this is the last layer
                    kwargs['return_sequences'] = False
                    model.add(LSTM(**kwargs))
            model.add(Dropout(dropout))
        model.add(Dense(OUTPUT_SIZE))
        model.add(Activation('softmax'))
    else:
        model, epoch = load_model_from_checkpoint(experiment_dir)

    # these cli args aren't specified if get_model() is being
    # being called from sample.py
    args2 = ["grad_clip","optimizer"]#dumb workaround for existing code
    if 'grad_clip' in args2 and 'optimizer' in args2:
        kwargs = { 'clipvalue': grad_clip }

        if learning_rate:
            kwargs['lr'] = learning_rate

        # select the optimizers
        if optimizer == 'sgd':
            optimizer = SGD(**kwargs)
        elif optimizer == 'rmsprop':
            optimizer = RMSprop(**kwargs)
        elif optimizer == 'adagrad':
            optimizer = Adagrad(**kwargs)
        elif optimizer == 'adadelta':
            optimizer = Adadelta(**kwargs)
        elif optimizer == 'adam':
            optimizer = Adam(**kwargs)
        elif optimizer == 'adamax':
            optimizer = Adamax(**kwargs)
        elif optimizer == 'nadam':
            optimizer = Nadam(**kwargs)
        else:
            log(
                'Error: {} is not a supported optimizer. Exiting.'.format(optimizer),
                True)
            exit(1)
    else: # so instead lets use a default (no training occurs anyway)
        optimizer = Adam()

    model.compile(loss='categorical_crossentropy', 
                  optimizer=optimizer,
                  metrics=['accuracy'])
    return model, epoch


def get_callbacks(experiment_dir, checkpoint_monitor='val_acc'):
    
    callbacks = []
    
    # save model checkpoints
    filepath = os.path.join(experiment_dir, 
                            'checkpoints', 
                            'checkpoint-epoch_{epoch:03d}-val_acc_{val_accuracy:.3f}.hdf5')
    print(filepath)

    callbacks.append(ModelCheckpoint(filepath = filepath, 
                                     monitor=checkpoint_monitor, 
                                     verbose=1, 
                                     save_best_only=False, 
                                     mode='max'))

    callbacks.append(ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.5, 
                                       patience=3, 
                                       verbose=1, 
                                       mode='auto', 
                                       epsilon=0.0001, 
                                       cooldown=0, 
                                       min_lr=0))

    callbacks.append(TensorBoard(log_dir=os.path.join(experiment_dir, 'tensorboard-logs'), 
                                histogram_freq=0, 
                                write_graph=True, 
                                write_images=False))

    return callbacks

def main():
    
    data_dir = "data/midi"
    verbose = True


    try:
        # get paths to midi files in --data_dir
        midi_files = [os.path.join(data_dir, path) \
                      for path in os.listdir(data_dir) \
                      if '.mid' in path or '.midi' in path]
    except OSError as e:
        log('Error: Invalid --data_dir, {} directory does not exist. Exiting.', verbose)
        exit(1)
    print(midi_files)
    log(
        'Found {} midi files in {}'.format(len(midi_files), "data/midi"),
        True
    )

    if len(midi_files) < 1:
        log(
            'Error: no midi files found in {}. Exiting.'.format("data/midi"),
            True
        )
        exit(1)

    # create the experiment directory and return its name
    experiment_dir = create_experiment_dir("experiments/default", True)

    # write --message to experiment_dir
    # if args.message:
    #     with open(os.path.join(experiment_dir, 'message.txt'), 'w') as f:
    #         f.write(args.message)
    #         log('Wrote {} bytes to {}'.format(len(args.message), 
    #             os.path.join(experiment_dir, 'message.txt')), args.verbose)

    val_split = 0.2 # use 20 percent for validation
    val_split_index = int(float(len(midi_files)) * val_split)

    window_size = 20
    batch_size = 32
    n_jobs = 1
    max_files_in_ram = 25
    num_epochs = 50
    verbose = True
    # use generators to lazy load train/validation data, ensuring that the
    # user doesn't have to load all midi files into RAM at once
    train_generator = get_data_generator(midi_files[0:val_split_index], 
                                               window_size=window_size,
                                               batch_size=batch_size,
                                               num_threads=n_jobs,
                                               max_files_in_ram=max_files_in_ram)

    val_generator = get_data_generator(midi_files[val_split_index:], 
                                             window_size=window_size,
                                             batch_size=batch_size,
                                             num_threads=n_jobs,
                                             max_files_in_ram=max_files_in_ram)

    model, epoch = get_model()
    if verbose:
        print(model.summary())

    save_model(model, experiment_dir)
    log('Saved model to {}'.format(os.path.join(experiment_dir, 'model.json')),
              verbose)

    callbacks = get_callbacks(experiment_dir)
    
    print('fitting model...')
    # this is a somewhat magic number which is the average number of length-20 windows
    # calculated from ~5K MIDI files from the Lakh MIDI Dataset.
    magic_number = 827
    start_time = time.time()
    model.fit_generator(train_generator,
                        steps_per_epoch=len(midi_files) * magic_number / batch_size, 
                        epochs=num_epochs,
                        validation_data=val_generator, 
                        validation_steps=len(midi_files) * 0.2 * magic_number / batch_size,
                        verbose=1, 
                        callbacks=callbacks,
                        initial_epoch=epoch)
    log('Finished in {:.2f} seconds'.format(time.time() - start_time), verbose)

main()

# Generating new midi files

In [0]:
import argparse, os, pdb
import pretty_midi


def get_experiment_dir(experiment_dir):
	
	if experiment_dir == 'experiments/default':
		dirs_ = [os.path.join('experiments', d) for d in os.listdir('experiments') \
		         if os.path.isdir(os.path.join('experiments', d))]
		experiment_dir = max(dirs_, key=os.path.getmtime)

	if not os.path.exists(os.path.join(experiment_dir, 'model.json')):
		log('Error: {} does not exist. ' \
			      'Are you sure that {} is a valid experiment?' \
			      'Exiting.'.format(os.path.join(args.experiment_dir), 'model.json',
			                        experiment_dir), True)
		exit(1)

	return experiment_dir

def main2(number_files, instrument, length_file):

    verbose = True
    data_dir = "data/midi"
    experiment_dir = "experiments/default"




    
    midi_files = [ os.path.join(data_dir, f) for f in os.listdir(data_dir) \
                 if '.mid' in f or '.midi' in f ]

    experiment_dir = get_experiment_dir(experiment_dir)
    log('Using {} as --experiment_dir'.format(experiment_dir), verbose)

    save_dir = os.path.join(experiment_dir, 'generated')

    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
        log('Created directory {}'.format(save_dir), verbose)
    
    model, epoch = get_model(experiment_dir=experiment_dir)
    print("test")
    log('Model loaded from {}'.format(os.path.join(experiment_dir, 'model.json')), 
              verbose)

    window_size = model.layers[0].get_input_shape_at(0)[1]
    seed_generator = get_data_generator(midi_files, 
                                              window_size=window_size,
                                              batch_size=32,
                                              num_threads=1,
                                              max_files_in_ram=10)

    midi_instrument = instrument
    file_length = length_file
    num_files = number_files

    # validate midi instrument name
    try:
    	# try and parse the instrument name as an int
    	instrument_num = int(midi_instrument)
    	if not (instrument_num >= 0 and instrument_num <=127):
          # print("error")
    		log('Error: {} is not a supported instrument. Number values must be ' \
    			      'be 0-127. Exiting'.format(midi_instrument), True)
    		exit(1)
    	midi_instrument = pretty_midi.program_to_instrument_name(instrument_num)
    except ValueError as err:
    	# if the instrument name is a string
    	try:
    		# validate that it can be converted to a program number
    		_ = pretty_midi.instrument_name_to_program(midi_instrument)
    	except ValueError as er:
        # print("error")
    		log('Error: {} is not a valid General MIDI instrument. Exiting.'\
    			      .format(midi_instrument), True)
    		exit(1)


    # log('Loading seed files...', verbose)
    X, y = next(seed_generator)
    generated = generate(model, X, window_size, 
    	                       file_length, num_files, midi_instrument)
    for i, midi in enumerate(generated):
        file = os.path.join(save_dir, '{}.mid'.format(i + 1))
        midi.write(file.format(i + 1))
        log('wrote midi file to {}'.format(file), True)

number_files = 10
instrument = "Acoustic Bass"
length_file = 1000
main2(number_files, instrument, length_file)

# Plotting graphs in tensorboard

In [0]:
%reload_ext tensorboard
# %load_ext tensorboard
%tensorboard --logdir experiments --port 6008